# Introduction to Natural Language Processing 2 Lab04

## Introduction

1. I want to choose the hate dataset because it is more violent than offensive. So it is more important to detect hate tweets. Furthermore if we sell it to a client, it is more impactful to describe our API as a hate eraser. Hate is easier to detect because an offense can be more subtile.

## Evaluating the dataset

In [1]:
!pip install datasets
!pip install bertopic

     |████████████████████████████████| 90 kB 339 kB/s eta 0:00:01
     |████████████████████████████████| 5.2 MB 1.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 514 kB/s  eta 0:00:01
     |████████████████████████████████| 85 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 5.5 MB 55.0 MB/s eta 0:00:01
     |████████████████████████████████| 182 kB 54.3 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 52.1 MB/s eta 0:00:01
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2444501 sha256=5010dbfdcc497e54b5821b41c6f6bbcd0e294b2e2ca37a6dcc4bd36e5f8522c9
  Stored in directory: /root/.cache/pip/wheels/93/78/2e/03ee191669a772e9653260aa3bd53e0b1a768751a9676e8c82
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ce97b

In [2]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "hate")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/490k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/hate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2970
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [4]:
print(dataset["train"]["label"].count(1))
print(dataset["train"]["label"].count(0))

3783
5217


In [5]:
dataset["train"][:2]

{'text': ['@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you…',
  'A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎'],
 'label': [0, 1]}

1. The dataset is split in three : the train, the test and the validation part. It is well proportionated, we have almost half hate tweets, half non-hate tweets. The hate tweets are labeled by a one and the non-hate tweets, by a zero. Non-hate tweets are just random texts generaly with people soft opinion.

2. all-MiniLM-L6-v2 is a sentence-transformers model. It maps sentences & paragraphs to a 384 dimensional dense vector space and is used for tasks like clustering or semantic search. We chose this model because it is easy to use and one of the fastest to perform topic search.

In [6]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP

umap_model = UMAP(random_state=42)
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(dataset["train"]["text"], show_progress_bar=True)
topic_model = BERTopic(umap_model=umap_model, language="english").fit(dataset["train"]["text"], embeddings)

2022-11-23 22:05:35.001924: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/282 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
topic = topic_model.get_topic(0)
print(topic)

[('rape', 0.06833998468866477), ('women', 0.02223294461451212), ('woman', 0.014699342464405256), ('men', 0.012902312132046566), ('raped', 0.01263521799020611), ('sexual', 0.012503162428152583), ('metoo', 0.012472435617503125), ('assault', 0.011605418008567663), ('weinstein', 0.010850334774138094), ('about', 0.010551735455915106)]


In [8]:
topic_model.visualize_topics()

3. The model has perfectly identified topics. We can observe topic clusters which constitute larger topics like racisme and misogyny. It will be harder to do the same on tweets from other countries. The differents languages and expressions will perturbate the model.

## Evaluate a model

In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from sklearn.metrics import f1_score, precision_score, recall_score
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

task='hate'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/588 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [13]:
scores_list = []
output_list = []

test = dataset["test"]

i = 0
true_positive = 0
false_negative = 0
true_negative = 0
false_positive = 0

for text in test["text"]:
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  if scores[0] > scores[1] and test[i]["label"] == 0:
    true_negative += 1
  elif scores[0] > scores[1] and test[i]["label"] == 1:
    false_positive += 1
  elif scores[0] < scores[1] and test[i]["label"] == 0:
    false_negative += 1
  elif scores[0] < scores[1] and test[i]["label"] == 1:
    true_positive += 1
  i+=1

In [17]:
recall = true_positive/(true_positive + false_negative)
print("recall : " + str(recall))
precision = true_positive/(true_positive + false_positive)
print("precision : "+ str(precision))
f1_score = (2 * precision * recall)/(precision + recall)
print("F1 score : " + str(f1_score))

recall : 0.4989491382934006
precision : 0.9480830670926518
F1 score : 0.6538143762049022
